In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

Using TensorFlow backend.


# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N


In [7]:
non_numbered = []
unique_val = {}
month_num = {
    'Jan-2019':1,
    'Feb-2019':2,
    'Mar-2019':3,
    'Apr-2019':4,
    'May-2019':5,
    'Jun-2019':6,
    'Jul-2019':7,
    'Aug-2019':8,
    'Sep-2019':9,
    'Oct-2019':10,
    'Nov-2019':11,
    'Dec-2019':12
}

#find columns that don't hold int data type so that they may be label encoded
for column in df:
    if df[column].dtypes == 'object':
        non_numbered.append(column)
print(non_numbered)

['home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type', 'hardship_flag', 'debt_settlement_flag']


In [9]:
#finding unique values to know which columns need to be label encoded
for column in non_numbered:
    unique_values = df[column].unique()
    count = len(unique_values)
    unique_val.update({column:count})
print(unique_val)

{'home_ownership': 4, 'verification_status': 3, 'issue_d': 3, 'loan_status': 2, 'pymnt_plan': 1, 'initial_list_status': 2, 'next_pymnt_d': 2, 'application_type': 2, 'hardship_flag': 1, 'debt_settlement_flag': 1}


In [11]:
#changing month-year format to an int based on the month
df['issue_d'] = df['issue_d'].apply(lambda x: month_num[x])
df['next_pymnt_d'] = df['next_pymnt_d'].apply(lambda x: month_num[x])

In [25]:
#label encoding the columns that need to be else binary encode ones that dont
from sklearn.preprocessing import LabelEncoder

label_en = LabelEncoder()

label_en.fit(df['home_ownership'])
df['home_ownership'] = label_en.transform(df['home_ownership'])

label_en.fit(df['loan_status'])
df['loan_status'] = label_en.transform(df['loan_status'])

label_en.fit(df['verification_status'])
df['verification_status'] = label_en.transform(df['verification_status'])

In [26]:
# binary encoding
binary = pd.get_dummies(df, columns=['pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'debt_settlement_flag'])
binary

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,dti,delinq_2yrs,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,3,66000.0,1,3,1,27.24,0.0,...,38199.0,2000.0,61987.0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,1,105000.0,2,3,1,20.23,0.0,...,60641.0,41200.0,49197.0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,1,56000.0,2,3,1,24.26,0.0,...,45684.0,7500.0,43144.0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,3,92000.0,2,3,1,31.44,0.0,...,68784.0,19700.0,76506.0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,1,52000.0,0,3,1,18.76,0.0,...,25919.0,27600.0,20000.0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,3,26000.0,1,1,1,9.60,0.0,...,6798.0,11300.0,5425.0,1,0,1,1,0,1,1
68813,12000.0,0.2727,368.37,3,63000.0,0,1,1,29.07,0.0,...,60350.0,13500.0,62939.0,1,0,1,1,0,1,1
68814,5000.0,0.1992,185.62,1,52000.0,1,1,1,14.86,0.0,...,18611.0,3600.0,18492.0,1,0,1,1,0,1,1
68815,40000.0,0.0646,1225.24,1,520000.0,2,1,1,9.96,0.0,...,95958.0,100800.0,78634.0,1,1,0,1,0,1,1


# Split the Data into Training and Testing

In [27]:
# Create our features
X = binary.drop(columns='loan_status')

# Create our target
y = binary['loan_status']

In [28]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,dti,delinq_2yrs,inq_last_6mths,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,1.726172,21.778153,0.217766,0.497697,...,6.133843e+04,29734.128558,5.572240e+04,1.0,0.123879,0.876121,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.743862,20.199244,0.718367,0.758122,...,5.738798e+04,26795.394232,5.095845e+04,0.0,0.329446,0.329446,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,1.000000,0.000000,0.000000,0.000000,...,2.350000e+02,100.000000,1.270000e+02,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,1.000000,13.890000,0.000000,0.000000,...,2.650300e+04,11600.000000,2.288000e+04,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,2.000000,19.760000,0.000000,0.000000,...,4.535700e+04,22100.000000,4.200000e+04,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,2.000000,26.660000,0.000000,1.000000,...,7.657000e+04,39300.000000,7.249900e+04,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,3.000000,999.000000,18.000000,5.000000,...,1.295455e+06,509400.000000,1.426964e+06,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [30]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [32]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, random_state=1, stratify=y)
X_train.shape

(51612, 87)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [33]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_sample, y_sample = ros.fit_resample(X_train, y_train)
Counter(y_sample)

Counter({1: 51352, 0: 51352})

In [35]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=1)

#fit
rf = rf.fit(X_sample, y_sample)

from sklearn.metrics import balanced_accuracy_score
y_predict = rf.predict(X_test)
balanced_accuracy_score(y_test, y_predict)

0.6605982410126868

In [36]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predict)

array([[   28,    59],
       [   11, 17107]], dtype=int64)

In [37]:
# Print the imbalanced classification report
# YOUR CODE HERE

from imblearn.metrics import classification_report_imbalanced

cri = classification_report_imbalanced(y_test, y_predict)
print(cri)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.72      0.32      1.00      0.44      0.57      0.30        87
          1       1.00      1.00      0.32      1.00      0.57      0.34     17118

avg / total       1.00      1.00      0.33      1.00      0.57      0.34     17205



In [39]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
df_sorted = rf.feature_importances_
sorted(zip(rf.feature_importances_, X.columns), reverse=True)

[(0.09511516189694809, 'total_rec_prncp'),
 (0.07648620716014995, 'total_rec_int'),
 (0.0635824126023575, 'last_pymnt_amnt'),
 (0.05837711676923794, 'total_pymnt'),
 (0.05548643389953452, 'issue_d'),
 (0.05479020749312779, 'total_pymnt_inv'),
 (0.02734869886184637, 'int_rate'),
 (0.019129313851143828, 'installment'),
 (0.018352687635554324, 'out_prncp_inv'),
 (0.01776559475254157, 'dti'),
 (0.01758559029523361, 'annual_inc'),
 (0.017511611792473836, 'out_prncp'),
 (0.016726849264744385, 'avg_cur_bal'),
 (0.014918974705195362, 'mo_sin_old_il_acct'),
 (0.014764677451760326, 'bc_open_to_buy'),
 (0.013826974899763134, 'il_util'),
 (0.013765139501049567, 'max_bal_bc'),
 (0.01346591146045051, 'bc_util'),
 (0.013314473658028805, 'mths_since_recent_inq'),
 (0.013252746449134543, 'tot_cur_bal'),
 (0.013177940378569732, 'revol_bal'),
 (0.013162157775538192, 'mo_sin_old_rev_tl_op'),
 (0.013090851802594108, 'total_rec_late_fee'),
 (0.013065867100262212, 'loan_amnt'),
 (0.012091565167460303, 'total

### Easy Ensemble AdaBoost Classifier

In [40]:
# Train the Classifier
# YOUR CODE HERE
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=100, random_state=1)

abc = abc.fit(X_sample, y_sample)

In [41]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_predict = abc.predict(X_test)
balanced_accuracy_score(y_test, y_predict)

0.9097552082703828

In [42]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_predict)

array([[   73,    14],
       [  335, 16783]], dtype=int64)

In [43]:
# Print the imbalanced classification report
# YOUR CODE HERE
cri = classification_report_imbalanced(y_test, y_predict)
print(cri)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.18      0.84      0.98      0.29      0.91      0.81        87
          1       1.00      0.98      0.84      0.99      0.91      0.83     17118

avg / total       1.00      0.98      0.84      0.99      0.91      0.83     17205

